##### nbappinator - first bqnt app

This app runs a BQL query that takes a single parameter, displays the result as an interactive table, and provides a simple charting interface that draws an interactive Plotly chart of the results.

#### Install libraries

This line will install any packages listed in the requirements.txt. 

In BQuant Enterprise, it's better/faster to use a Custom Environment and add packages to it. This avoids having to install the requirements every time you start BQuant. 

In [ ]:
%pip install -r requirements.txt

##### Import your packages

In [ ]:
import nbappinator as nbapp
import jinja2
import logging
import bql
import pandas as pd
import plotly.express as px

#### Setup any globals

Setup static variables used in the global scope (outside a function) that will be used without the function.

Try to only use static globals: global variables that don't change / aren't mutated.

In [ ]:
logging.basicConfig(encoding='utf-8', level=logging.INFO)

logger = logging.getLogger(__name__)

bq = bql.Service()
    
INITIAL_QUERY = """get(
      px_last
    ) for(
      '{{SECURITY}}'
    ) with(
      dates=range(-29d, 0d),
      fill=prev
    )"""

PAGES = ["Data", "Chart"]


#### Define functions

Functions to execute the BQL query, apply the template, and draw a chart.

In [ ]:
def _exec_bql(query: str) -> pd.DataFrame:
    r = bq.execute(query)
    return bql.combined_df(r)

def execute_query(component: str, action: str, args: str, app: nbapp.UiModel, caller: str):
    with app.messages:

        # Create query from a template
        security: str = app.get_values("security") 
        base_query: str = app.get_values("query") 
        query = jinja2.Template(base_query).render(SECURITY=security)
        print(query)

        # Execute the query
        df = _exec_bql(query)

        # Display the dataframe in a grid
        target_page = app.get_page(PAGES[0]) 
        target_page.clear_page()
        target_page.add_df(name="df1", df=df)


        # Create a menu on the chart_page that lets the user pick columns from the chart.
        chart_page = app.get_page(PAGES[1]) 
        chart_page.clear_page()
        cols = list(df.columns)
        chart_page.add_box(name="options", horiz=True)
        chart_page.add_select(override_page = "options", name="x_axis", label="X Axis", options=cols, value="DATE")
        chart_page.add_select(override_page = "options", name="y_axis", label="Y Axis", options=cols, value=cols[-1])
        chart_page.add_select(override_page = "options", name="z_axis", label="Series", options=cols, value="ID")
        chart_page.add_button(name="drawchart", label="Draw Chart", action=draw_chart, status=False)
        chart_page.add_container(name="chartcontainer")


def draw_chart(component: str, action: str, args: str, app: nbapp.UiModel, caller: str):
    with app.messages:
        df = app.widgets["df1"].w.df
        print(df.columns)

        x = app.get_values("x_axis")
        y = app.get_values("y_axis")
        color = app.get_values("z_axis")

        fig = px.line(df, x=x, y=y, color=color)
        
        chart_page = app.get_page(PAGES[1])
        app.clear_container("chartcontainer") 
        chart_page.add_plotly_fig(name="f1", fig=fig, override_page="chartcontainer")


# Setup and Display the nbappinator Application

In [ ]:
myapp = nbapp.TabbedUiModel(pages=PAGES, log_footer = "Messages", headers=["Config"])

config_page = myapp.get_page("Config")
config_page.add_textfield(name="security", label="Enter Security", value="IBM US Equity")
config_page.add_textarea(name="query", label="Enter BQL, with {SECURITY} for the Universe", value=INITIAL_QUERY)

config_page.add_button(name="update", label="Run Query", action=execute_query, status=False)


myapp.display()